In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv("../Data/faa_ai_prelim.csv")

print("\n--- Dataset Info ---")
print(df.info())

print("\n--- First 5 Rows ---")
display(df.head())


--- Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UPDATED              83 non-null     object 
 1   ENTRY_DATE           83 non-null     object 
 2   EVENT_LCL_DATE       83 non-null     object 
 3   EVENT_LCL_TIME       83 non-null     object 
 4   LOC_CITY_NAME        83 non-null     object 
 5   LOC_STATE_NAME       83 non-null     object 
 6   LOC_CNTRY_NAME       2 non-null      object 
 7   RMK_TEXT             83 non-null     object 
 8   EVENT_TYPE_DESC      83 non-null     object 
 9   FSDO_DESC            81 non-null     object 
 10  REGIST_NBR           83 non-null     object 
 11  FLT_NBR              6 non-null      object 
 12  ACFT_OPRTR           7 non-null      object 
 13  ACFT_MAKE_NAME       78 non-null     object 
 14  ACFT_MODEL_NAME      79 non-null     object 
 15  ACFT_MISSING_FLAG   

,UPDATED,ENTRY_DATE,EVENT_LCL_DATE,EVENT_LCL_TIME,LOC_CITY_NAME,LOC_STATE_NAME,LOC_CNTRY_NAME,RMK_TEXT,EVENT_TYPE_DESC,FSDO_DESC,...,PAX_INJ_NONE,PAX_INJ_MINOR,PAX_INJ_SERIOUS,PAX_INJ_FATAL,PAX_INJ_UNK,GRND_INJ_NONE,GRND_INJ_MINOR,GRND_INJ_SERIOUS,GRND_INJ_FATAL,GRND_INJ_UNK
0,No,19-FEB-16,19-FEB-16,00:45:00Z,MARSHVILLE,North Carolina,NaN,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",Accident,FAA Charlotte FSDO-68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,No,19-FEB-16,18-FEB-16,23:55:00Z,TAVERNIER,Florida,NaN,AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...,Incident,FAA Miami FSDO-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,No,19-FEB-16,18-FEB-16,22:14:00Z,TRENTON,New Jersey,NaN,"AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...",Incident,FAA Philadelphia FSDO-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,19-FEB-16,18-FEB-16,17:10:00Z,ASHEVILLE,North Carolina,NaN,"AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...",Incident,FAA Charlotte FSDO-68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,No,19-FEB-16,18-FEB-16,00:26:00Z,TALKEETNA,Alaska,NaN,"AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...",Incident,FAA Anchorage FSDO-03,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['EVENT_DATETIME'] = pd.to_datetime(
    df['EVENT_LCL_DATE'] + ' ' + df['EVENT_LCL_TIME'],
    errors='coerce'
)

C:\Users\ishat\AppData\Local\Temp\ipykernel_20096\1976979467.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['EVENT_DATETIME'] = pd.to_datetime(


In [4]:
required_columns = [
    'ACFT_MAKE_NAME',     # Aircraft make name
    'LOC_STATE_NAME',     # State name
    'ACFT_MODEL_NAME',    # Aircraft model name
    'RMK_TEXT',           # Text information
    'FLT_PHASE',          # Flight phase
    'EVENT_TYPE_DESC',    # Event description type
    'FATAL_FLAG',         # Fatal flag
    'ACFT_DMG_DESC',      # Aircraft damage description
    'EVENT_DATETIME'      # Combined datetime column
]
df_req = df[required_columns].copy()

In [5]:
df_req['FATAL_FLAG'] = df_req['FATAL_FLAG'].fillna('No')

# Fill missing FLT_PHASE and ACFT_DMG_DESC with mode values
for col in ['FLT_PHASE', 'ACFT_DMG_DESC']:
    mode_val = df_req[col].mode()[0]
    df_req[col] = df_req[col].fillna(mode_val)

# Verify missing values replaced
print("\n--- Missing Values After Filling ---")
print(df_req.isnull().sum())


--- Missing Values After Filling ---
ACFT_MAKE_NAME     5
LOC_STATE_NAME     0
ACFT_MODEL_NAME    4
RMK_TEXT           0
FLT_PHASE          0
EVENT_TYPE_DESC    0
FATAL_FLAG         0
ACFT_DMG_DESC      0
EVENT_DATETIME     0
dtype: int64


In [6]:
original_count = len(df_req)
print(f"Original number of observations: {original_count}")

Original number of observations: 83


In [7]:
df_cleaned = df_req.dropna(subset=['ACFT_MAKE_NAME'])

In [8]:
cols_to_drop = [col for col in df.columns if df[col].isnull().sum() > 75]
df_cleaned = df_cleaned.drop(columns=[c for c in cols_to_drop if c in df_cleaned.columns])

after_count = len(df_cleaned)
print(f"Number of observations after cleaning: {after_count}")
print(f"Columns dropped: {cols_to_drop}")

Number of observations after cleaning: 78
Columns dropped: ['LOC_CNTRY_NAME', 'FLT_NBR', 'ACFT_OPRTR', 'ACFT_MISSING_FLAG', 'FAR_PART', 'FLT_CRW_INJ_MINOR', 'FLT_CRW_INJ_SERIOUS', 'FLT_CRW_INJ_UNK', 'CBN_CRW_INJ_NONE', 'CBN_CRW_INJ_MINOR', 'CBN_CRW_INJ_SERIOUS', 'CBN_CRW_INJ_FATAL', 'CBN_CRW_INJ_UNK', 'PAX_INJ_MINOR', 'PAX_INJ_SERIOUS', 'PAX_INJ_FATAL', 'PAX_INJ_UNK', 'GRND_INJ_NONE', 'GRND_INJ_MINOR', 'GRND_INJ_SERIOUS', 'GRND_INJ_FATAL', 'GRND_INJ_UNK']


In [9]:
aircraft_counts = df_cleaned.groupby('ACFT_MAKE_NAME').size().sort_values(ascending=False)
display(aircraft_counts)

ACFT_MAKE_NAME
CESSNA                    23
PIPER                     10
BEECH                      9
MOONEY                     4
BOEING                     3
CHAMPION                   2
SOCATA                     2
LANCAIR                    2
BELL                       2
CHRISTEN                   1
AEROSTAR INTERNATIONAL     1
AIRBUS                     1
AERO COMMANDER             1
AERONCA                    1
FAIRCHILD                  1
ENSTROM                    1
EMBRAER                    1
CONSOLIDATED VULTEE        1
GRUMMAN                    1
FLIGHT DESIGN              1
GREAT LAKES                1
GLOBE                      1
MAULE                      1
HUGHES                     1
GULFSTREAM                 1
NORTH AMERICAN             1
PITTS                      1
SAAB                       1
SABRELINER                 1
VANS                       1
dtype: int64

In [10]:
fatal_cases = df_cleaned[df_cleaned['FATAL_FLAG'] == 'Yes']
display(fatal_cases)

,ACFT_MAKE_NAME,LOC_STATE_NAME,ACFT_MODEL_NAME,RMK_TEXT,FLT_PHASE,EVENT_TYPE_DESC,FATAL_FLAG,ACFT_DMG_DESC,EVENT_DATETIME
0,BEECH,North Carolina,36,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",UNKNOWN (UNK),Accident,Yes,Unknown,2016-02-19 00:45:00+00:00
53,PIPER,Florida,PA28,AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES. ...,UNKNOWN (UNK),Accident,Yes,Destroyed,2016-02-11 11:32:00+00:00
55,FLIGHT DESIGN,California,CTLS,AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES A...,UNKNOWN (UNK),Accident,Yes,Destroyed,2016-02-11 00:17:00+00:00
79,NORTH AMERICAN,Arizona,F51,"AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...",UNKNOWN (UNK),Accident,Yes,Destroyed,2016-02-05 11:17:00+00:00
80,CHAMPION,California,8KCAB,"N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...",UNKNOWN (UNK),Accident,Yes,Destroyed,2016-02-05 23:02:00+00:00
81,BEECH,California,35,"N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...",UNKNOWN (UNK),Accident,Yes,Destroyed,2016-02-05 23:02:00+00:00
82,CESSNA,Alabama,182,N784CP AIRCRAFT CRASHED INTO A WOODED AREA NEA...,UNKNOWN (UNK),Accident,Yes,Unknown,2016-02-02 01:52:00+00:00


In [11]:
df_encoded = pd.get_dummies(df_cleaned, columns=['ACFT_DMG_DESC'], drop_first=True)

print("\n--- Encoded Dataset Head ---")
display(df_encoded.head())


--- Encoded Dataset Head ---


,ACFT_MAKE_NAME,LOC_STATE_NAME,ACFT_MODEL_NAME,RMK_TEXT,FLT_PHASE,EVENT_TYPE_DESC,FATAL_FLAG,EVENT_DATETIME,ACFT_DMG_DESC_Minor,ACFT_DMG_DESC_Substantial,ACFT_DMG_DESC_Unknown
0,BEECH,North Carolina,36,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",UNKNOWN (UNK),Accident,Yes,2016-02-19 00:45:00+00:00,False,False,True
1,VANS,Florida,RV7,AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...,LANDING (LDG),Incident,No,2016-02-18 23:55:00+00:00,False,False,True
2,CESSNA,New Jersey,172,"AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...",APPROACH (APR),Incident,No,2016-02-18 22:14:00+00:00,True,False,False
3,LANCAIR,North Carolina,235,"AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...",LANDING (LDG),Incident,No,2016-02-18 17:10:00+00:00,False,False,True
4,CESSNA,Alaska,172,"AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...",LANDING (LDG),Incident,No,2016-02-18 00:26:00+00:00,True,False,False


In [12]:
df_encoded.to_csv("faa_cleaned_encoded.csv", index=False)
print("Final cleaned & encoded dataset saved as 'faa_cleaned_encoded.csv'")

Final cleaned & encoded dataset saved as 'faa_cleaned_encoded.csv'


In [13]:
phase_keywords = {
    'LANDING': 'LANDING',
    'TAKEOFF': 'TAKEOFF',
    'CRUISE': 'CRUISE',
    'APPROACH': 'APPROACH'
}

def extract_flight_phase(text):
    if pd.isna(text):
        return 'UNKNOWN'
    text_upper = str(text).upper()
    for keyword, phase in phase_keywords.items():
        if keyword in text_upper:
            return phase
    return 'UNKNOWN'

df['FLIGHT_PHASE_TEXT'] = df['RMK_TEXT'].apply(extract_flight_phase)

comparison = pd.crosstab(df['FLT_PHASE'], df['FLIGHT_PHASE_TEXT'], dropna=False)

print("\n--- Crosstab Comparison between FLT_PHASE and FLIGHT_PHASE_TEXT ---")
print(comparison)

matches = (df['FLT_PHASE'].str.upper() == df['FLIGHT_PHASE_TEXT']).sum()
print(f"\nExact matches: {matches} out of {len(df)} rows")


--- Crosstab Comparison between FLT_PHASE and FLIGHT_PHASE_TEXT ---
FLIGHT_PHASE_TEXT  APPROACH  LANDING  TAKEOFF  UNKNOWN
FLT_PHASE                                             
APPROACH (APR)            1        0        0        1
LANDING (LDG)             0       36        0       11
STANDING (STD)            0        0        0        3
TAKEOFF (TOF)             0        0        5        2
TAXI (TXI)                0        1        0        9
UNKNOWN (UNK)             0        0        0       13
NaN                       0        0        0        1

Exact matches: 0 out of 83 rows
